In [18]:

import pandas as pd
import plotly.express as px
import altair as alt
# from sqlalchemy import create_engine
# import secrets
import numpy as np
import datetime

In [19]:
df = pd.read_csv(r"data/all_sales_data.csv")
# ### MASTER DATA ###
all_sales = df.copy()

all_sales = all_sales[all_sales.market_segment != 'Samples']

# invoice date cleanup
all_sales['date'] = pd.to_datetime(all_sales['date'])
all_sales['date'] = all_sales['date'].dt.normalize()
all_sales['date'] = all_sales['date'].dt.floor('D')

In [23]:
current_date = datetime.today.strftime('%Y-%m-%d')
# import datetime
year_ago_today = datetime.today - datetime.timedelta(days=365)
# year_ago_today.strftime('%Y-%m-%d')

# ---- TOP KPI's Row ----
sales_24 = int(all_sales[(all_sales['date'] > '2023-12-31') & (all_sales['date'] < current_date)].usd.sum())
sales_23 = int(all_sales[(all_sales['date'] > '2022-12-31') & (all_sales['date'].dt.date < year_ago_today.date())].usd.sum())
delta = sales_24 - sales_23

df_selection = all_sales.copy()

AttributeError: module 'datetime' has no attribute 'today'

In [14]:
print(sales_23)
print(sales_24)
print(current_date)
print(year_ago_today)
print(datetime.datetime.today())

353784
448033
2024-01-26
2023-01-26 21:52:59.488306
2024-01-26 21:55:26.633468


In [4]:
############

market_segment_color = {
    'Vending': 'rgb(56,149,73)',
    'Grocery': 'rgb(248,184,230)',
    'Alternate Retail': 'rgb(46,70,166)',
    'Canada': 'rgb(204,208,221)',
    'Online': 'rgb(106,87,63)',
    'Other': 'rgb(200,237,233)',
    'Convenience': 'rgb(233,81,46)',
    'Broadline Distributor': 'rgb(233,152,19)',
    'Samples': 'rgb(141,62,92)'}

In [ ]:
df_selection.index = pd.to_datetime(df_selection['date'],format = '%m/%d/%y')
mth_sales = round(df_selection.groupby(pd.Grouper(freq='M'))['usd'].sum())

In [16]:


seg_sales = round(df_selection.groupby('market_segment',as_index=False)['usd'].sum()).sort_values(by='market_segment',ascending=False)
fig_seg_sales = px.pie(
    seg_sales, 
    values='usd', 
    names='market_segment',
    template = 'simple_white',
    title=' ',
    opacity = .8,
    hole=.33,
    hover_data = ['market_segment'],
    color='market_segment',
    color_discrete_map=market_segment_color).update_layout(autosize=False,width=450,height=450,showlegend=False)
fig_seg_sales.update_traces(textposition='inside', textinfo='percent+label')#,texttemplate='%{percent:.0%}')
fig_seg_sales.show()

In [26]:
df_selection.index = pd.to_datetime(df_selection['date'],format = '%y/%m/%d')

sales_per_week = df_selection.groupby(pd.Grouper(freq='W'))['usd'].sum()

In [27]:
sales_per_week

date
2021-01-03        78.4800
2021-01-10     34765.2375
2021-01-17     68371.7175
2021-01-24    170547.5400
2021-01-31    145100.5875
                 ...     
2023-12-31    188384.2750
2024-01-07    133904.8450
2024-01-14     85863.3750
2024-01-21    110610.7275
2024-01-28    117654.1475
Freq: W-SUN, Name: usd, Length: 161, dtype: float64

In [46]:
#all sales weekly scatter
fig_scatter_all = px.scatter(
    round(sales_per_week),
    x=sales_per_week.index,
    y='usd',
    title='<b>Weekly Sales</b>',
    template = 'presentation',
    labels={'date':'',
            'usd':'<b>$USD</b>'},
    height=425,
    size='usd',
    size_max=15,
    color='usd',
    color_continuous_scale=px.colors.sequential.Oranges,
    trendline="rolling", trendline_options=dict(function="mean", window=10), trendline_scope="overall", trendline_color_override="black"
)

fig_scatter_all.update_coloraxes(showscale=False)
fig_scatter_all.update_yaxes(gridcolor='lightgray')

fig_scatter_all.update_layout(
    title_x=0.4,
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        # title = '10 Week Moving Average'
    )
)

fig_scatter_all.update_traces(
    hovertemplate=
    "Sales: <b>%{y:$,.0f}")

fig_scatter_all.show()